In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [15]:
inf_data = pd.read_csv('Final_Dataset.csv')

In [16]:
inf_data.head(5)

,DATE,CPI,M2,UNRATE,DSPI,PCE,FEDFUNDS,INDPRO,M1SL
0,01-01-1990,132.1,3166.8,5.4,4208.4,3730.7,8.23,61.6440,795.4
1,01-02-1990,132.7,3179.2,5.3,4236.3,3728.2,8.24,62.2108,798.1
2,01-03-1990,133.5,3190.1,5.2,4254.0,3754.9,8.28,62.5068,801.5
3,01-04-1990,134.0,3201.6,5.4,4290.8,3770.0,8.26,62.3597,806.1
4,01-05-1990,134.4,3200.6,5.4,4296.0,3775.8,8.18,62.5474,804.2


In [17]:
df_inf = inf_data[['DATE', 'CPI']].copy()


In [18]:
df_inf.head(5)

,DATE,CPI
0,01-01-1990,132.1
1,01-02-1990,132.7
2,01-03-1990,133.5
3,01-04-1990,134.0
4,01-05-1990,134.4


In [19]:
# Convert the 'date' column to datetime format
df_inf['DATE'] = pd.to_datetime(df_inf['DATE'])



In [20]:
df_inf['year'] = pd.to_datetime(df_inf['DATE']).dt.year

In [21]:
df_inf.head()

,DATE,CPI,year
0,1990-01-01,132.1,1990.0
1,1990-01-02,132.7,1990.0
2,1990-01-03,133.5,1990.0
3,1990-01-04,134.0,1990.0
4,1990-01-05,134.4,1990.0


In [22]:
# Create an interactive line plot
fig = px.line(df_inf, x='DATE', y='CPI', title='US Inflation Rate Trends (1947-2023)',
              labels={'Date': 'Year', 'CPI': 'Inflation Rate'},
              template='plotly', line_shape='linear')

# Customize the x-axis date format
fig.update_xaxes(tickformat='%Y')

# Show the plot
fig.show()

In [23]:
# Extract year and inflation rate
df_inf['year'] = df_inf['DATE'].dt.year
data = df_inf[['year', 'CPI']]


In [24]:
# Set the 'year' column as the index
data.set_index('year', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [25]:
# Split data into train and test sets
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

In [26]:
# Create sequences for LSTM training
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12  # Length of sequences for LSTM
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [27]:
# Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
10/10 [==============================] - 6s 10ms/step - loss: 0.2117
Epoch 2/100
10/10 [==============================] - 0s 9ms/step - loss: 0.0171
Epoch 3/100
10/10 [==============================] - 0s 8ms/step - loss: 0.0145
Epoch 4/100
10/10 [==============================] - 0s 9ms/step - loss: 0.0041
Epoch 5/100
10/10 [==============================] - 0s 9ms/step - loss: 0.0035
Epoch 6/100
10/10 [==============================] - 0s 8ms/step - loss: 0.0018
Epoch 7/100
10/10 [==============================] - 0s 9ms/step - loss: 0.0012
Epoch 8/100
10/10 [==============================] - 0s 8ms/step - loss: 7.2327e-04
Epoch 9/100
10/10 [==============================] - 0s 8ms/step - loss: 3.9259e-04
Epoch 10/100
10/10 [==============================] - 0s 9ms/step - loss: 2.0828e-04
Epoch 11/100
10/10 [==============================] - 0s 9ms/step - loss: 1.1301e-04
Epoch 12/100
10/10 [==============================] - 0s 9ms/step - loss: 6.0135e-05
Epoch 13/100
10/

In [28]:
# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

3/3 [==============================] - 0s 8ms/step


In [29]:
# Inverse transform predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform(y_test)

In [30]:
# Create a DataFrame for plotting
plot_data = pd.DataFrame({
    'Year': np.concatenate((data.index[seq_length:seq_length+len(train_predict)],
                            data.index[seq_length+len(train_predict):seq_length+len(train_predict)+len(test_predict)])),
    'Predicted CPI': np.concatenate((train_predict.flatten(), test_predict.flatten())),
    'Actual CPI': np.concatenate((y_train.flatten(), y_test.flatten()))
})

# Create an interactive line plot using Plotly Express
fig = px.line(plot_data, x='Year', y=['Predicted CPI', 'Actual CPI'], title='CPI Forecasting with LSTM')
fig.update_layout(xaxis_title='Year', yaxis_title='CPI', legend_title='Data')
fig.show()

In [33]:
# Calculate Mean Squared Error for the training set
mse_train = mean_squared_error(y_train, train_predict)
print(f'Mean Squared Error (Train): {mse_train}')



Mean Squared Error (Train): 0.2604622306559557


The model's performance was evaluated by comparing its predictions to the actual inflation rates. The model's predictions were reasonably accurate, with a mean squared error of 0.26.

The results of this project show that LSTM models can be used to forecast inflation with a reasonable degree of accuracy. However, it is important to note that inflation is a complex phenomenon and there are many factors that can affect it. Therefore, it is important to use multiple forecasting methods to get a more accurate picture of future inflation rates.

The results of this project can be used to better understand the factors that drive inflation and to develop better strategies for forecasting inflation. This information can be used by businesses and governments to make informed decisions about their financial planning.